In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pickle
import librosa
import pandas as pd
import numpy as np
import IPython.display as ipd
import os
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le = LabelEncoder()

import warnings
warnings.filterwarnings("ignore")

In [ ]:
ls

drive/  sample_data/


In [ ]:
os.getcwd()

'/content'

In [ ]:
ls /content/drive/MyDrive

In [ ]:
# os.path.exists("/content/drive/MyDrive/dataset.zip")

In [ ]:
# !unzip /content/drive/MyDrive/dataset.zip -d '/content/drive/MyDrive'

In [ ]:
os.path.exists("/content/drive/MyDrive/final")

True

In [ ]:
os.chdir("/content/drive/MyDrive/final")

In [ ]:
ls

data1.pkl  data_spec_128.pkl  lug_aug_flat.tflite  model.png  train/
data.pkl   data_spec.pkl      lug.tflite           test/


In [ ]:
len(os.listdir("train")),len(os.listdir("test"))

(20, 20)

# extract file names and labels

In [ ]:
train_files = []
train_labels= []
for i in tqdm(os.listdir("train")):
#     print(i)
    for x in os.listdir(os.path.join("train",i)):
        x = os.path.join("train",i,x)
        label = i
        train_files.append(x)
        train_labels.append(label)

# Splitting the data into training and validation sets
train_files, val_files, train_labels, val_labels = train_test_split(train_files, train_labels, test_size=0.11, random_state=42)
len(train_files),len(val_files)

100%|██████████| 20/20 [00:13<00:00,  1.49it/s]


(8127, 1005)

In [ ]:
test_files = []
test_labels= []
for i in tqdm(os.listdir("test")):
#     print(i)
    for x in os.listdir(os.path.join("test",i)):
        x = os.path.join("test",i,x)
        label = i
        test_files.append(x)
        test_labels.append(label)

100%|██████████| 20/20 [00:02<00:00,  8.09it/s]


In [ ]:
len(test_files),len(train_files),len(val_files)

(711, 8127, 1005)

# extract features

In [ ]:
def add_white_noise(signal, noise_percentage_factor):
    noise = np.random.normal(0, signal.std(), signal.size)
    augmented_signal = signal + noise * noise_percentage_factor
    return augmented_signal


def time_stretch(signal, time_stretch_rate):
    """Time stretching implemented with librosa:
    https://librosa.org/doc/main/generated/librosa.effects.pitch_shift.html?highlight=pitch%20shift#librosa.effects.pitch_shift
    """
    return librosa.effects.time_stretch(signal, time_stretch_rate)


def pitch_scale(signal, sr, num_semitones):
    """Pitch scaling implemented with librosa:
    https://librosa.org/doc/main/generated/librosa.effects.pitch_shift.html?highlight=pitch%20shift#librosa.effects.pitch_shift
    """
    return librosa.effects.pitch_shift(signal, sr = sr, n_steps = num_semitones)


def random_gain(signal, min_factor=0.1, max_factor=0.12):
    gain_rate = random.uniform(min_factor, max_factor)
    augmented_signal = signal * gain_rate
    return augmented_signal


def invert_polarity(signal):
    return signal * -1

In [ ]:
def extract_mfccs(signal,sr):
#     audio, sr = librosa.load(path, sr=16000)
#     print("loading ..........")
#     signal = librosa.util.fix_length(audio, size=sr*4)
#     print("resizing.............")
    mfcc_features = librosa.feature.mfcc(y=signal, sr = sr, n_mfcc=13, n_fft=2048, fmin = 100,hop_length=512)
    mfc = mfcc_features.T
    mfc = np.expand_dims(mfc, axis=-1)
    return mfc

def load(path):
    audio, sr = librosa.load(path, sr=16000)
    signal = librosa.util.fix_length(audio, size=sr*4)
    return signal,sr

def augment(signal):
    a = add_white_noise(signal,0.2)
    b = pitch_scale(signal,sr=16000,num_semitones=2)
    return signal,a,b

In [ ]:
c = []
path = test_files[4]
lab = []
signal,sr = load(path)
x = augment(signal)
for a in x:
    c.append(extract_mfccs(a,sr))
    lab.append(test_labels[2])
c = np.array(c)
c.shape, lab

((3, 126, 13, 1), ['door close', 'door close', 'door close'])

In [ ]:
%%time
X_test = []
for x in tqdm(test_files):
    signal,sr = load(x)
    mfc = extract_mfccs(signal,sr)
    X_test.append(mfc)
Xtest = np.array(X_test)
ytest = le.fit_transform(test_labels)

Xtest.shape,ytest.shape, ytest[:67]

100%|██████████| 711/711 [06:41<00:00,  1.77it/s]

CPU times: user 25.6 s, sys: 1min 9s, total: 1min 35s
Wall time: 6min 41s


((711, 126, 13, 1),
 (711,),
 array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 7,
        7]))

In [ ]:
%%time
X_val = []
for x in tqdm(val_files):
    signal,sr = load(x)
    mfc = extract_mfccs(signal,sr)
    X_val.append(mfc)
Xval = np.array(X_val)
Xval.shape

100%|██████████| 1005/1005 [09:08<00:00,  1.83it/s]

CPU times: user 35.7 s, sys: 1min 35s, total: 2min 11s
Wall time: 9min 8s


(1005, 126, 13, 1)

In [ ]:
%%time
X_train = []
train_labels_= []
i = 0
for x in tqdm(train_files):
    signal,sr = load(x)
    y = augment(signal)
    for a in y:
        X_train.append(extract_mfccs(a,sr))
        train_labels_.append(train_labels[i])
    i= i+1
Xtrain = np.array(X_train)
Xtrain.shape

100%|██████████| 8127/8127 [1:25:51<00:00,  1.58it/s]


CPU times: user 15min 23s, sys: 16min 19s, total: 31min 43s
Wall time: 1h 25min 51s


(24381, 126, 13, 1)

In [ ]:
le.classes_

array(['alarm off', 'alarm on', 'camera off', 'camera on', 'door close',
       'door open', 'fan decrease', 'fan increase', 'fan off', 'fan on',
       'fridge off', 'fridge on', 'lights off', 'lights on',
       'speaker decrease', 'speaker increase', 'speaker off',
       'speaker on', 'tv off', 'tv on'], dtype='<U16')

In [ ]:
ytrain = le.transform(train_labels_)
yval = le.transform(val_labels)
ytrain.shape, yval.shape

((24381,), (1005,))

In [ ]:
len(ytrain)

24381

In [ ]:
train_labels[:200]

['alarm on',
 'tv off',
 'camera on',
 'alarm off',
 'lights off',
 'tv off',
 'tv on',
 'tv on',
 'fridge on',
 'speaker off',
 'tv on',
 'fan increase',
 'camera on',
 'tv on',
 'fan on',
 'speaker on',
 'fridge on',
 'door open',
 'tv on',
 'tv on',
 'door close',
 'camera off',
 'door close',
 'speaker increase',
 'speaker on',
 'tv off',
 'door close',
 'fan on',
 'tv on',
 'alarm off',
 'alarm on',
 'speaker decrease',
 'speaker on',
 'alarm on',
 'fan increase',
 'door open',
 'fan on',
 'fridge on',
 'fridge on',
 'alarm on',
 'tv off',
 'speaker on',
 'door close',
 'speaker on',
 'speaker on',
 'alarm on',
 'fan off',
 'alarm off',
 'alarm off',
 'speaker increase',
 'fan off',
 'speaker decrease',
 'alarm off',
 'speaker off',
 'speaker on',
 'tv on',
 'alarm on',
 'tv on',
 'speaker increase',
 'camera off',
 'fridge on',
 'speaker off',
 'fridge off',
 'speaker on',
 'fridge off',
 'speaker increase',
 'fan decrease',
 'fan on',
 'camera off',
 'speaker decrease',
 'door c

In [ ]:
ytrain[:300]

array([ 1,  1,  1, 18, 18, 18,  3,  3,  3,  0,  0,  0, 12, 12, 12, 18, 18,
       18, 19, 19, 19, 19, 19, 19, 11, 11, 11, 16, 16, 16, 19, 19, 19,  7,
        7,  7,  3,  3,  3, 19, 19, 19,  9,  9,  9, 17, 17, 17, 11, 11, 11,
        5,  5,  5, 19, 19, 19, 19, 19, 19,  4,  4,  4,  2,  2,  2,  4,  4,
        4, 15, 15, 15, 17, 17, 17, 18, 18, 18,  4,  4,  4,  9,  9,  9, 19,
       19, 19,  0,  0,  0,  1,  1,  1, 14, 14, 14, 17, 17, 17,  1,  1,  1,
        7,  7,  7,  5,  5,  5,  9,  9,  9, 11, 11, 11, 11, 11, 11,  1,  1,
        1, 18, 18, 18, 17, 17, 17,  4,  4,  4, 17, 17, 17, 17, 17, 17,  1,
        1,  1,  8,  8,  8,  0,  0,  0,  0,  0,  0, 15, 15, 15,  8,  8,  8,
       14, 14, 14,  0,  0,  0, 16, 16, 16, 17, 17, 17, 19, 19, 19,  1,  1,
        1, 19, 19, 19, 15, 15, 15,  2,  2,  2, 11, 11, 11, 16, 16, 16, 10,
       10, 10, 17, 17, 17, 10, 10, 10, 15, 15, 15,  6,  6,  6,  9,  9,  9,
        2,  2,  2, 14, 14, 14,  4,  4,  4,  7,  7,  7,  1,  1,  1, 17, 17,
       17, 16, 16, 16,  9

# save pickle file

In [ ]:
data = {
    'Xtrain': Xtrain,
    'Xtest': Xtest,
    'Xval': Xval,
    'ytrain': ytrain,
    'ytest': ytest,
    'yval': yval
}

In [ ]:
# Specify the path to save the pickle file
pickle_path = 'data13.pkl'

with open(pickle_path, 'wb') as file:
    pickle.dump(data, file)

In [ ]:
pickle_path = 'data13.pkl'

with open(pickle_path, 'rb') as file:
    data2 = pickle.load(file)

Xtrain1 = data2['Xtrain']
Xtest1 = data2['Xtest']
Xval1 = data2['Xval']
ytrain1 = data2['ytrain']
ytest1 = data2['ytest']
yval1 = data2['yval']

Xtrain1.shape,Xtest1.shape

((24381, 126, 13, 1), (711, 126, 13, 1))